# **Imports**

In [1]:
import os
os.chdir("../../..")
os.chdir(r"src")

from Polarization.word_partisanship import *
from GloVe.weights import *
from Axes.projection_functions import *
from Axes.axes_definition import *
from Axes.models import *

import numpy as np

import warnings
warnings.filterwarnings("ignore")

os.chdir("../")

/Users/alexandrequeant/Desktop/Travail-TSE


# **Polarization log odds ratio**

## **Unigrams**

In [2]:
party_1 = "Lab"
party_2 = "Con" 

In [3]:
dfs_partisanship = []

for i in range(10):
    df = standard_opening(
            "data/FinalDataframes/FilteredFinalDataFrame_201" + str(i) + ".csv", True
        )
    df = df[df["source"] == "par"]
    year = eval("201" + str(i))
    partisanship_matrix, idx2words, words2idx = get_word_partisanship(df, year, party_1, party_2)
    deltas = partisanship_matrix[3, :]
    words = np.array(list(idx2words.values()), dtype=None)
    df_sorted_partisanships = pd.DataFrame({'words' : words, 'deltas' : deltas}).sort_values(by='deltas', ascending=False)
    dfs_partisanship.append(df_sorted_partisanships)

In [5]:
dfs_partisanship[7]

,words,deltas
12749,curlew,0.562265
14111,primat,0.562261
5799,quarri,0.562261
419,veolia,0.562261
9855,torr,0.562260
...,...,...
11136,bach,-0.824050
12667,karim,-0.824050
9494,threadbar,-0.824050
9741,superrich,-0.824051


## **Bigrams**

In [2]:
party_1 = "Lab"
party_2 = "Con" 

In [25]:
dfs_partisanship = []

for i in range(10):
    df = standard_opening(
            "data/FinalDataframes/FilteredFinalDataFrame_201" + str(i) + ".csv", True
        )
    df = df[df["source"] == "par"]
    year = eval("201" + str(i))
    partisanship_matrix, idx2words, words2idx = get_word_partisanship(df, year, party_1, party_2, bigram=True)
    deltas = partisanship_matrix[3, :]
    words = np.array(list(idx2words.values()), dtype=None)
    df_sorted_partisanships = pd.DataFrame({'words' : words, 'deltas' : deltas}).sort_values(by='deltas', ascending=False)
    dfs_partisanship.append(df_sorted_partisanships)

# **Combine with embeddings**

In [3]:
def cosine_with_axis_word(word: str, b,  model_words) :   

    array_1 = model_words[word]
    array_2 = b

    cosine = np.dot(array_1, array_2.T) / (norm(array_1) * norm(array_2))

    return cosine

def cosine_with_axis_bigram(bigram: str, b,  model_words) :   

    word_1, word_2 = bigram.split(' ')

    array_1 = (model_words[word_1] + model_words[word_2]) * 0.5
    array_2 = b

    cosine = np.dot(array_1, array_2.T) / (norm(array_1) * norm(array_2))

    return cosine

def get_quantiles(data, percentiles):
    """
    Get quantiles from a distribution.
    
    Parameters:
        data (array-like): The data.
        percentiles (array-like): The percentiles to compute (0-100).
    
    Returns:
        quantiles (array): The values at the specified percentiles.
    """
    return np.percentile(data, percentiles)

def filter_deltas(df, delta_low_percentile, delta_high_percentile):
    delta_percentiles = [delta_low_percentile, delta_high_percentile]

    quantiles_deltas = get_quantiles(df['deltas'], delta_percentiles)

    df = df.loc[(df['deltas'] < quantiles_deltas[0]) | (df['deltas'] > quantiles_deltas[1])]

    return df

## For unigrams

In [77]:
for i in range(10) :

    df = dfs_partisanship[i]
    m = models_w[i]
    pos_a = filter_model(pos_1, m)
    neg_a = filter_model(neg_1, m)

    b = barycentre(pos_a, m) - barycentre(neg_a, m)
    df['cos'] = df['words'].apply(cosine_with_axis_word, b=b, model_words=m)
    dfs_partisanship[i] = df

## For bigrams

In [28]:
for i in range(10) :

    df = dfs_partisanship[i]
    m = models_w[i]
    pos_a = filter_model(pos_1, m)
    neg_a = filter_model(neg_1, m)

    b = barycentre(pos_a, m) - barycentre(neg_a, m)
    df['cos'] = df['words'].apply(cosine_with_axis_bigram, b=b, model_words=m)
    dfs_partisanship[i] = df

## Filtering

In [79]:
df = dfs_partisanship[7]

In [65]:
df = filter_deltas(df, 10, 90, 20, 0)

In [ ]:
df.sort_values(by='deltas', ascending=False)

In [8]:
len(models_w)

14

In [19]:
df_proj = pd.read_csv("data/current_dataframes/df_BT")
year = 2017
df_par = df_proj.loc[df_proj['year'] == year]

In [27]:
def get_quantiles(data, percentiles):
    """
    Compute quantiles for a given dataset and percentiles.
    
    :param data: Numerical data from which to calculate quantiles.
    :param percentiles: A list of percentiles to calculate for the data.
    :return: An array of quantiles corresponding to the specified percentiles.
    """
    return np.percentile(data, percentiles)

def partizan_words(left_side, right_side, year, gram = 'bigram', focus_on_companies=None, axis=None,
    percentiles_cos=[10, 90], percentiles_delta=[10, 90], force_i_lim=None, re_filter_cos=False, percentiles_refiltering_cos = [25, 75]):

    sources = left_side+right_side

    s = str(year)[-2:]

    if s[0] == '1':
        i = eval(s[1])
    if s[0] == '2':
        i = eval('1'+s[1])
    
    df_proj = pd.read_csv("data/current_dataframes/df_BT")

    df_par = df_proj.loc[df_proj["source"].isin(sources) | df_proj["party"].isin(sources)]
    
    def change_year(old_year):
        if int(old_year) == 20110:
            return 2020
        if int(old_year) == 20111:
            return 2021
        if int(old_year) == 20112:
            return 2022
        if int(old_year) == 20113:
            return 2023
        else :
            return int(old_year)
        
    df_par['year'] = df_par['year'].apply(change_year)
    df_par = df_par.loc[df_par['year'] == year]

    df1 = df_par[df_par['source'] == 'par']
    df2 = df_par[df_par['source'] != 'par']

    # Define a function to translate newspaper source to party
    def translate_party(newspaper):
        """
        Translates newspaper sources to their corresponding political party.
        
        :param newspaper: The source to be translated.
        :return: The political party corresponding to the source.
        """
        if newspaper in left_side:
            return "Lab"
        if newspaper in right_side:
            return "Con"
    
    # Apply the translation function to assign parties based on sources
    df2['party'] = df2['source'].apply(translate_party)
    df2['Speaker'] = range(len(df2))

    # Combine the two DataFrames and reset index for continuity
    df_par = pd.concat([df1, df2]).reset_index(drop=True)

    if focus_on_companies:
        df_par = df_par.loc[df_par["class"].isin(focus_on_companies)]

    if axis is not None:
        quantiles = get_quantiles(df_par[f'cos axe {axis}'], percentiles_cos)
        df_par = df_par[(df_par[f'cos axe {axis}'] < quantiles[0]) | (df_par[f'cos axe {axis}'] > quantiles[1])]

    def phrase_to_tokens(phrase):
        word_list = phrase.strip("_").split("_")
        return word_list
    
    df_par['text'] = df_par['text'].apply(phrase_to_tokens)

    if gram == 'bigram' :
        partisanship_matrix, idx2words, words2idx = get_word_partisanship(df_par, year, 'Lab', 'Con', bigram=True)
        deltas = partisanship_matrix[3, :]
        words = np.array(list(idx2words.values()), dtype=None)
        df_sorted_partisanships = pd.DataFrame({'words' : words, 'deltas' : deltas}).sort_values(by='deltas', ascending=False)
    

        df = df_sorted_partisanships
        m = models_w[i]
        pos_a = filter_model(pos_1, m)
        neg_a = filter_model(neg_1, m)

        b = barycentre(pos_a, m) - barycentre(neg_a, m)
        df['cos'] = df['words'].apply(cosine_with_axis_bigram, b=b, model_words=m)
        df_sorted_partisanships = df
        df = filter_deltas(df, percentiles_delta[0], percentiles_delta[1])

    if gram == 'unigram' :
        partisanship_matrix, idx2words, words2idx = get_word_partisanship(df_par, year, 'Lab', 'Con', bigram=False)
        deltas = partisanship_matrix[3, :]
        words = np.array(list(idx2words.values()), dtype=None)
        df_sorted_partisanships = pd.DataFrame({'words' : words, 'deltas' : deltas}).sort_values(by='deltas', ascending=False)
        
        df = df_sorted_partisanships
        m = models_w[i]
        pos_a = filter_model(pos_1, m)
        neg_a = filter_model(neg_1, m)

        b = barycentre(pos_a, m) - barycentre(neg_a, m)
        df['cos'] = df['words'].apply(cosine_with_axis_word, b=b, model_words=m)
        df_sorted_partisanships = df
        df = filter_deltas(df, percentiles_delta[0], percentiles_delta[1])

    if re_filter_cos:
        quantiles = get_quantiles(df['cos'], percentiles_refiltering_cos)
        df = df[(df['cos'] < quantiles[0]) | (df['cos'] > quantiles[1])]

    return df.sort_values(by='deltas', ascending=False)

In [29]:
partizan_words(['Lab'], ['Con'], 2017, focus_on_companies=['fb', 'ap'])

      Unnamed: 0                                               text  \
2153        78.0  [absolut, heart, point, make, peopl, need, tim...   
2154        97.0  [tribut, work, sixthform, educ, fantast, work,...   
2155       106.0  [much, welcom, comment, absolut, metaphor, nee...   
2156       170.0  [permiss, would, like, make, statement, polit,...   
2157       282.0  [longer, term, peopl, consum, less, sugar, thi...   
...          ...                                                ...   
5206     22232.0  [honour, serv, join, choru, welldeserv, congra...   
5207     22236.0  [concur, comment, absolut, right, mere, word, ...   
5208     22243.0  [certainli, appli, case, good, point, remain, ...   
5209     22247.0  [make, interest, suggest, take, away, come, ba...   
5210     22248.0  [grate, debat, thank, respond, mani, point, ma...   

                                     sentence_embedding source party  \
2153  ['-0.03473206079978031', '0.05706717554553675'...    par   Con   
215

,words,deltas,cos
2261,fulltim franchis,0.741466,0.013489
864,call bass,0.741466,0.108509
10603,shelter london,0.741466,0.043616
12791,prefer follow,0.741466,0.008373
9946,plan stcenturi,0.741466,0.028350
12425,abroad understand,0.741466,0.007615
16109,alik deliv,0.741466,0.039942
18950,wrist struck,0.741466,-0.081348
17291,assur commun,0.741466,-0.110624
14720,press burma,0.741466,-0.066298
